In [240]:
import pandas as pd
from datetime import datetime, timedelta
import datetime as dt
import requests
import zipfile as zf
import StringIO
import re
import os
import numpy as np
import sys
from os.path import join, isfile, abspath
from os import listdir
import matplotlib.pyplot as plt
from math import radians, cos, sin, asin, sqrt

%pylab inline

Populating the interactive namespace from numpy and matplotlib


/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['sample', 'radians', 'sin', 'datetime', 'cos', 'sqrt']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [241]:
two_up= abspath(join(os.getcwd(),"../.."))
calls_path = join(two_up,"share/data/calls")
listdir = os.listdir(calls_path)
dirs = [x for x in listdir if isfile(join(calls_path,x))==False]
dirs = [join(calls_path,x) for x in dirs]
allfiles = []
for d in dirs:
    listdir = os.listdir(d)
    paths = [join(d, x) for x in listdir]
    allfiles  = allfiles + paths
allfiles = [x for x in allfiles if x.split('.')[1]=='csv']


In [242]:
# trips and stops
trips_path = join(two_up,"share/data/trips")
trips = pd.read_csv(join(trips_path, "all_trips.csv"))
stops_path = join(two_up,"share/data/stops")
stops = pd.read_csv(join(stops_path, "all_stops.csv"))

In [245]:
# params
today_str = '20170501'
today = dt.datetime.strptime(today_str,'%Y%m%d')
tomorrow = today + timedelta(days=1)

In [244]:
fname0 = allfiles[0]
fname1 = allfiles[1]
df0 = pd.read_csv(fname0)
df1 = pd.read_csv(fname1)

In [ ]:
calls = pd.concat([df0, df1]).sort_values(['vehicle_id','timestamp'])
calls = calls[calls.service_date == int(today_str)]
calls.head()

# start

In [150]:
last_date = '0000-00-00'
last_trip = -1
last_vehicle = 0
last_stop = -1
last_time = dt.datetime.now()
columns = ['vehicle_id','service_date','trip_id','next_stop_id','timestamp']
init = pd.DataFrame(pd.Series([last_vehicle, last_date, last_trip, last_stop, last_time],
                              columns)).T

In [151]:
block_assigned = calls.block_assigned
calls = calls[columns]

In [152]:
calls = init.append(calls)
calls.index = range(len(calls))

In [153]:
calls['next_next_stop_id'] = calls.next_stop_id.tolist()[1:] + [-1]
calls['next_time'] = calls.timestamp.tolist()[1:] + [np.nan]

In [154]:
calls['comp'] = calls.next_stop_id == calls.next_next_stop_id

In [155]:
calls1 = calls[calls.comp==True].copy()
calls2 = calls[calls.comp==False].copy()

In [156]:
calls1['dep_start'] = calls1['next_time']
calls2['dep_start'] = calls2['timestamp']

In [157]:
calls1['dep_end'] = np.nan
calls2['dep_end'] = calls2['next_time']

In [158]:
calls = pd.concat([calls1,calls2])

In [159]:
calls=calls.sort_index().iloc[:-1,:]

In [160]:
calls['block_assigned'] = block_assigned.tolist()
calls.head()

,vehicle_id,service_date,trip_id,next_stop_id,timestamp,next_next_stop_id,next_time,comp,dep_start,dep_end,block_assigned
0,0,0000-00-00,-1,-1,2017-06-17 19:46:37.885674,551045,2017-05-01T11:20:09Z,False,2017-06-17 19:46:37.885674,2017-05-01T11:20:09Z,1
1,185,20170501,15372217-CPPB7-CP_B7-Weekday-10-SDon,551045,2017-05-01T11:20:09Z,551045,2017-05-01T11:20:41Z,True,2017-05-01T11:20:41Z,NaN,1
2,185,20170501,15372217-CPPB7-CP_B7-Weekday-10-SDon,551045,2017-05-01T11:20:41Z,551036,2017-05-01T11:21:12Z,False,2017-05-01T11:20:41Z,2017-05-01T11:21:12Z,1
3,185,20170501,15372217-CPPB7-CP_B7-Weekday-10-SDon,551036,2017-05-01T11:21:12Z,551046,2017-05-01T11:22:16Z,False,2017-05-01T11:21:12Z,2017-05-01T11:22:16Z,1
4,185,20170501,15372217-CPPB7-CP_B7-Weekday-10-SDon,551046,2017-05-01T11:22:16Z,551047,2017-05-01T11:23:52Z,False,2017-05-01T11:22:16Z,2017-05-01T11:23:52Z,1


In [161]:
calls = calls.dropna()

# merged

In [162]:
merged = pd.merge(calls, trips, how='left', on='trip_id')

In [163]:
merged = merged[['service_date','trip_id','vehicle_id','next_stop_id',
                'dep_start','dep_end','route_id','direction_id','block_assigned']]

In [165]:
merged = merged.dropna()
merged.head(10)

,service_date,trip_id,vehicle_id,next_stop_id,dep_start,dep_end,route_id,direction_id,block_assigned
1,20170501,15372217-CPPB7-CP_B7-Weekday-10-SDon,185,551045,2017-05-01T11:20:41Z,2017-05-01T11:21:12Z,Q25,1.0,1
2,20170501,15372217-CPPB7-CP_B7-Weekday-10-SDon,185,551036,2017-05-01T11:21:12Z,2017-05-01T11:22:16Z,Q25,1.0,1
3,20170501,15372217-CPPB7-CP_B7-Weekday-10-SDon,185,551046,2017-05-01T11:22:16Z,2017-05-01T11:23:52Z,Q25,1.0,1
4,20170501,15372217-CPPB7-CP_B7-Weekday-10-SDon,185,551047,2017-05-01T11:24:23Z,2017-05-01T11:25:59Z,Q25,1.0,1
5,20170501,15372217-CPPB7-CP_B7-Weekday-10-SDon,185,551026,2017-05-01T11:25:59Z,2017-05-01T11:27:35Z,Q25,1.0,1
6,20170501,15372217-CPPB7-CP_B7-Weekday-10-SDon,185,551048,2017-05-01T11:27:35Z,2017-05-01T11:29:38Z,Q25,1.0,1
7,20170501,15372217-CPPB7-CP_B7-Weekday-10-SDon,185,551019,2017-05-01T11:29:38Z,2017-05-01T11:31:43Z,Q25,1.0,1
8,20170501,15372217-CPPB7-CP_B7-Weekday-10-SDon,185,553130,2017-05-01T11:31:43Z,2017-05-01T11:33:51Z,Q25,1.0,1
9,20170501,15372217-CPPB7-CP_B7-Weekday-10-SDon,185,551015,2017-05-01T11:33:51Z,2017-05-01T11:35:59Z,Q25,1.0,1
10,20170501,15372217-CPPB7-CP_B7-Weekday-10-SDon,185,551013,2017-05-01T11:35:59Z,2017-05-01T11:37:34Z,Q25,1.0,1


In [166]:
def convert_utc(x):
    return dt.datetime.strptime(x, "%Y-%m-%dT%H:%M:%SZ") - timedelta(hours=5)
merged['dep_start'] = merged['dep_start'].apply(convert_utc)
merged['dep_end'] = merged['dep_end'].apply(convert_utc)


In [176]:
merged['timestamp'] = merged.dep_start +  (merged.dep_end-merged.dep_start)/2

In [181]:
merged = merged.sort_values(['next_stop_id','route_id','timestamp'])[merged.next_stop_id != '\N']

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


In [186]:
merged = merged[['trip_id','next_stop_id','route_id','direction_id','block_assigned','timestamp']]

In [188]:
merged['next_route'] = merged.route_id.tolist()[1:] + [np.nan]

In [190]:
merged['next_time'] = merged.timestamp.tolist()[1:] + [np.nan]

In [197]:
merged['next_next_stop'] = merged.next_stop_id.tolist()[1:] + [np.nan]

In [192]:
merged['comp'] = merged.route_id == merged.next_route

In [198]:
merged['comp2'] = merged.next_next_stop == merged.next_stop_id

In [205]:
merged['comp3'] = merged['comp'] & merged['comp2']

In [207]:
merged = merged[merged.comp3==True]

In [209]:
merged.drop(['next_route','comp','next_next_stop','comp2','comp3'], axis=1, inplace=True)

In [210]:
merged.head()

,trip_id,next_stop_id,route_id,direction_id,block_assigned,timestamp,next_time
429255,GH_B7-Weekday-SDon-039100_BX26_107,100014,BX26,1.0,1,2017-05-01 06:12:19.000,2017-05-01 06:21:36.500
1461163,GH_B7-Weekday-SDon-041700_BX238_9,100017,BX28,0.0,1,2017-05-01 06:09:36.500,2017-05-01 06:20:14.000
1461164,GH_B7-Weekday-SDon-041700_BX238_9,100018,BX28,0.0,1,2017-05-01 06:11:30.500,2017-05-01 06:21:48.500
633092,KB_B7-Weekday-SDon-042700_BX1_110,100019,BX1,0.0,1,2017-05-01 06:09:34.000,2017-05-01 06:14:52.000
896069,KB_B7-Weekday-SDon-043300_BX1_109,100019,BX1,0.0,1,2017-05-01 06:14:52.000,2017-05-01 06:23:02.000


In [215]:
merged['headway'] = merged.next_time - merged.timestamp

In [217]:
merged['headway'] = merged.headway.dt.total_seconds()/60

In [218]:
merged.head()

,trip_id,next_stop_id,route_id,direction_id,block_assigned,timestamp,next_time,headway
429255,GH_B7-Weekday-SDon-039100_BX26_107,100014,BX26,1.0,1,2017-05-01 06:12:19.000,2017-05-01 06:21:36.500,9.291667
1461163,GH_B7-Weekday-SDon-041700_BX238_9,100017,BX28,0.0,1,2017-05-01 06:09:36.500,2017-05-01 06:20:14.000,10.625000
1461164,GH_B7-Weekday-SDon-041700_BX238_9,100018,BX28,0.0,1,2017-05-01 06:11:30.500,2017-05-01 06:21:48.500,10.300000
633092,KB_B7-Weekday-SDon-042700_BX1_110,100019,BX1,0.0,1,2017-05-01 06:09:34.000,2017-05-01 06:14:52.000,5.300000
896069,KB_B7-Weekday-SDon-043300_BX1_109,100019,BX1,0.0,1,2017-05-01 06:14:52.000,2017-05-01 06:23:02.000,8.166667


In [226]:
merged['rds_index'] = merged.route_id.astype(str) + '_' + merged.direction_id.astype(int).astype(str) + '_' + merged.next_stop_id.astype(str)

In [227]:
merged.head()

,trip_id,next_stop_id,route_id,direction_id,block_assigned,timestamp,next_time,headway,rds_index
429255,GH_B7-Weekday-SDon-039100_BX26_107,100014,BX26,1.0,1,2017-05-01 06:12:19.000,2017-05-01 06:21:36.500,9.291667,BX26_1_100014
1461163,GH_B7-Weekday-SDon-041700_BX238_9,100017,BX28,0.0,1,2017-05-01 06:09:36.500,2017-05-01 06:20:14.000,10.625000,BX28_0_100017
1461164,GH_B7-Weekday-SDon-041700_BX238_9,100018,BX28,0.0,1,2017-05-01 06:11:30.500,2017-05-01 06:21:48.500,10.300000,BX28_0_100018
633092,KB_B7-Weekday-SDon-042700_BX1_110,100019,BX1,0.0,1,2017-05-01 06:09:34.000,2017-05-01 06:14:52.000,5.300000,BX1_0_100019
896069,KB_B7-Weekday-SDon-043300_BX1_109,100019,BX1,0.0,1,2017-05-01 06:14:52.000,2017-05-01 06:23:02.000,8.166667,BX1_0_100019


In [230]:
merged2 = merged[['rds_index','trip_id','timestamp','headway']]

In [231]:
merged2.head()

,rds_index,trip_id,timestamp,headway
429255,BX26_1_100014,GH_B7-Weekday-SDon-039100_BX26_107,2017-05-01 06:12:19.000,9.291667
1461163,BX28_0_100017,GH_B7-Weekday-SDon-041700_BX238_9,2017-05-01 06:09:36.500,10.625000
1461164,BX28_0_100018,GH_B7-Weekday-SDon-041700_BX238_9,2017-05-01 06:11:30.500,10.300000
633092,BX1_0_100019,KB_B7-Weekday-SDon-042700_BX1_110,2017-05-01 06:09:34.000,5.300000
896069,BX1_0_100019,KB_B7-Weekday-SDon-043300_BX1_109,2017-05-01 06:14:52.000,8.166667


In [251]:
merged2.rename(columns={'headway':'headways'}, inplace=True)

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/pandas/core/frame.py:2754: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


In [252]:
merged2.to_csv(join(two_up,'share','observed_headways_20170501_2.csv'), index=False)

In [234]:
len(merged2.trip_id.unique())

52556

In [253]:
merged.sort_values('headway', ascending=False).head(10)

,trip_id,next_stop_id,route_id,direction_id,block_assigned,timestamp,next_time,headway,rds_index
921146,MQ_B7-Weekday-SDon-012000_M14AD_2,404329,M14A,0.0,1,2017-05-01 00:58:42.500,2017-05-01 23:43:08.500,1364.433333,M14A_0_404329
641225,GH_B7-Weekday-SDon-004000_BX12_1,103795,BX12,1.0,1,2017-04-30 23:43:30.500,2017-05-01 21:42:09.500,1318.650000,BX12_1_103795
635561,GH_B7-Weekday-SDon-008000_BX12_2,101739,BX12,1.0,1,2017-05-01 00:25:20.000,2017-05-01 21:25:43.500,1260.391667,BX12_1_101739
921274,MQ_B7-Weekday-SDon-027000_M14AD_2,405265,M14A,1.0,1,2017-05-01 03:52:07.000,2017-05-01 23:54:43.000,1202.600000,M14A_1_405265
921277,MQ_B7-Weekday-SDon-027000_M14AD_2,405157,M14A,1.0,1,2017-05-01 03:54:14.500,2017-05-01 23:29:07.000,1174.875000,M14A_1_405157
921275,MQ_B7-Weekday-SDon-027000_M14AD_2,405152,M14A,1.0,1,2017-05-01 03:52:55.000,2017-05-01 23:27:16.000,1174.350000,M14A_1_405152
457410,CS_B7-Weekday-SDon-030000_Q48_2,804121,Q48,1.0,1,2017-05-01 04:28:04.000,2017-05-01 23:41:42.000,1153.633333,Q48_1_804121
824165,MQ_B7-Weekday-SDon-029300_M14AD_6,404286,M14A,0.0,1,2017-05-01 03:55:31.500,2017-05-01 23:08:26.000,1152.908333,M14A_0_404286
824164,MQ_B7-Weekday-SDon-029300_M14AD_6,401507,M14A,0.0,1,2017-05-01 03:54:43.000,2017-05-01 23:05:14.500,1150.525000,M14A_0_401507
641338,GH_B7-Weekday-SDon-021000_BX12_1,402611,BX12,1.0,1,2017-05-01 03:10:51.000,2017-05-01 22:03:38.000,1132.783333,BX12_1_402611


In [269]:
merged.to_csv("merged_observed_20170501.csv")

# read schedule

In [263]:
sched = pd.read_csv(join(two_up,'share','schedules','monday.csv'))

In [266]:
sched = sched.sort_values(['rds_index','arrival_time'])

In [267]:
sched.head()

,trip_id,arrival_time,stop_id,stop_sequence,pickup_type,drop_off_type,route_id,direction_id,service_id,rds_index,headways
0,16359052-SCPC7-SC_C7-Sunday-20,1900-01-01 00:13:08,300226,19,0,0,B100,0,SCPC7-SC_C7-Sunday-20,B100_0_300226,29.166667
1,16357241-SCPC7-SC_C7-Weekday-20,1900-01-01 06:13:58,300226,19,0,0,B100,0,SCPC7-SC_C7-Weekday-20,B100_0_300226,360.833333
2,16357255-SCPC7-SC_C7-Weekday-20,1900-01-01 06:43:58,300226,19,0,0,B100,0,SCPC7-SC_C7-Weekday-20,B100_0_300226,30.000000
3,16357272-SCPC7-SC_C7-Weekday-20,1900-01-01 06:58:58,300226,19,0,0,B100,0,SCPC7-SC_C7-Weekday-20,B100_0_300226,15.000000
4,16357280-SCPC7-SC_C7-Weekday-20,1900-01-01 07:05:58,300226,19,0,0,B100,0,SCPC7-SC_C7-Weekday-20,B100_0_300226,7.000000


In [268]:
merged3 = merged2.sort_values(['rds_index','timestamp'])
merged3.head()

,rds_index,trip_id,timestamp,headways
111468,B100_0_300226,15406510-SCPB7-SC_B7-Weekday-20-SDon,2017-05-01 05:15:13.000,10.966667
130363,B100_0_300226,15406510-SCPB7-SC_B7-Weekday-20-SDon,2017-05-01 05:26:11.000,19.025000
133032,B100_0_300226,15406524-SCPB7-SC_B7-Weekday-20-SDon,2017-05-01 05:45:12.500,8.216667
112952,B100_0_300226,15406524-SCPB7-SC_B7-Weekday-20-SDon,2017-05-01 05:53:25.500,11.583333
113733,B100_0_300226,15406542-SCPB7-SC_B7-Weekday-20-SDon,2017-05-01 06:05:00.500,21.825000
